In [1]:
from pathlib import Path
from dotenv import dotenv_values, load_dotenv

import torch
import coref_ds
from coref_ds.text import Text
from coref_ds.mmax.mmax_doc import MmaxDoc
from coref_ds.tei.tei_doc import TEIDocument
from coref_ds.tei.mention import Mention

from plotkarzyna.model import get_model, predict
from plotkarzyna.utils import get_head

try:
    local_config = dotenv_values(".env")
except FileNotFoundError:
    local_config = dict()
load_dotenv()

[nltk_data] Downloading package punkt to /home/ksaputa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
n_samples = 100_000_000
test_paths = list(
    (Path(local_config['PCC_ROOT']) / 'test').iterdir()
    )[:n_samples]

In [3]:
tei_docs = []
for p in test_paths:
    try: 
        m = TEIDocument(p)
        tei_docs.append(m)
    except Exception as e:
        print(e)

In [4]:
checkpoint_path = '/home/ksaputa/mspace/plotkarzyna/models/herbert-large-2/checkpoint-6240'
tokenizer, model = get_model(checkpoint_path)

In [5]:
preds = []
for ind, doc in enumerate(tei_docs):
    print(ind, end=' ')
    pred = predict(doc.text.segments, model, tokenizer)
    preds.append(pred)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 

In [6]:
import spacy
# spacy.cli.download('pl_core_news_lg')

In [7]:
nlp = spacy.load("pl_core_news_lg")

In [8]:
get_head(['Ala', 'ma', 'kota', '.'], nlp)

1

In [9]:
pred_path = Path(local_config['PCC_ROOT']) / 'plotkarzyna_herbert_large_preds'
for doc, pred in zip(tei_docs, preds):
    doc.layers['mentions'].remove_mentions()
    for ind, span in enumerate(pred[-2]):
        start, end = span
        segments = doc.text.segments[start:(end)]
        head = get_head(segments, nlp)
        try:
            doc.layers['mentions'].add_mention(
                Mention(
                    id=f"mention_{ind}",
                    text=' '.join(segments),
                    segments=segments,
                    span_start=start,
                    span_end=end,
                    lemmatized_text=None,
                    head_orth=None,
                    head=start + head,
                    cluster_id=None,
                ),
                doc.text.segments_meta
            )
        except:
            print(doc.doc_path)
            print(head, start, len(doc.text.segments_meta), segments, doc.text.segments)
    doc.to_file(pred_path / doc.doc_path.name)

/home/ksaputa/mspace/dataset/dataset/PCC-1.5-TEI-split/test/579
0 317 319 ['obecności', 'lekarza'] ['Gabinety', 'bez', 'lekarzy', 'Jeden', 'lekarz', '"', 'na', 'dochodne', '"', 'i', 'niecałe', 'sześć', 'pielęgniarek', '(', 'tylko', 'pięć', 'i', 'pół', 'etatu', ')', 'na', '7', 'tysięcy', 'uczniów', '-', 'tak', 'w', 'zgierskich', 'szkołach', 'wygląda', 'opieka', 'medyczna', '.', 'Innej', 'nie', 'będzie', ',', 'bo', 'medycyna', 'szkolna', 'jest', 'na', 'szarym', 'końcu', 'długiej', 'listy', 'potrzeb', '.', 'Lekarz', 'i', 'pielęgniarki', 'objechać', 'muszą', 'w', 'tygodniu', '17', 'miejskich', 'szkół', '.', 'Tylko', 'dwa', 'razy', 'w', 'tygodniu', 'pielęgniarka', 'jest', 'w', 'szkolnym', 'gabinecie', 'lekarskim', 'przez', '8', 'godzin', '.', '-', 'Standardy', 'szkolnej', 'medycyny', 'mówią', 'o', '800', 'uczniach', 'przypadających', 'na', 'jedną', 'pielęgniarkę', '.', 'W', 'Zgierzu', 'przypada', 'na', 'każdą', 'z', 'nas', 'aż', '1300', 'dzieciaków', '-', 'mówi', 'Aleksandra', 'Kwaśniewska'

In [10]:
for mention in preds[-1][-1]:
    print(' '.join(mention))

Katowicach
zespołów mieszkaniowych
15 placów budowy zespołów mieszkaniowych
budowy zespołów mieszkaniowych
katowickim Osiedlu Ptasim
Firma Holdimex , budująca na katowickim Osiedlu Ptasim
nowe ulice
13-hektarowy teren , na którym powstają nowe ulice
zrealizowała
400 mieszkań : przy ul Zimorodków , Gawronów , Kormoranów , Drozdów
ul Zimorodków , Gawronów , Kormoranów , Drozdów
Zimorodków , Gawronów , Kormoranów , Drozdów
przygotowaniu
ulica Pliszek
Pliszek
Drugie tyle
Holdimex
Osiedlu Ptasim
Osiedlu Ptasim
urozmaiconej architekturze
trzy- lub pięcio piętrowych budynkach o urozmaiconej architekturze
pięcio piętrowych budynkach o urozmaiconej architekturze
urozmaiconej architekturze
tu
cierpliwość
szybkim tempie
mieszkania , które są jeszcze na papierze , a będą gotowe dopiero w 2008 r .
papierze , a będą gotowe dopiero w 2008 r . Ich cena
dotychczasowej
tak zwanym stanie deweloperskim
metr kwadratowy w tak zwanym stanie deweloperskim
białego montażu
drzwi wewnętrznych
pokryć podłogowych


In [11]:
pred = preds[-1]
tokens, offset = pred[0], pred[-1]

In [12]:
offset[0]

['Katowicach']